In [5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pylab as plt
import csv
import math
%matplotlib inline

from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz


from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier
from sklearn_extensions.extreme_learning_machines.random_layer import RBFRandomLayer, MLPRandomLayer

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.neural_network import MLPClassifier

##################
# PREPARING DATA #
##################




def make_binary_label(name_part, dir_name, user_num, not_user_nums):
    portion_max_size = 90000
    last_num = 0    
    label = np.zeros((portion_max_size))    
    data = pd.read_csv(dir_name + str(user_num) + '.csv', header=None)    
    dim = data.shape[1]
    data_matr = data.as_matrix()
    data_ar = np.zeros((portion_max_size, dim))
    portions_num = data.shape[0]
    last_num = data.shape[0]
    for i in range(0, last_num):
        label[i] = 1
        data_ar[i] = data_matr[i]
        
    for i in not_user_nums:
        data = pd.read_csv(dir_name + str(i) + '.csv', header=None)
        data_matr = data.as_matrix()
        size = data.shape[0]
        for j in range(0, size):
            label[last_num] = 0
            data_ar[last_num] = data_matr[j]
            last_num = last_num + 1
        portions_num = portions_num + size
    
    label.resize(portions_num, 1)
    data_ar.resize(portions_num, dim) 
    with open("temp/data_auth_" + name_part + ".csv", 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for i in range(0, portions_num):
            writer.writerow(data_ar[i])
    
    with open("temp/label_auth_" + name_part + ".csv", 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        for i in range(0, portions_num):
            writer.writerow(label[i])
    




def spoil_data(data, persentage):
    size = data.shape[0]
    
    spoilt_size = int(size * persentage)
    inds = np.arange(size)
    np.random.shuffle(inds)
    for i in range(spoilt_size):
        data.values[inds[i]] = 0
       
    return data

directory = "extracted_features/"
make_binary_label("all", directory, 1, [2, 3, 4, 5, 6]) # choosing the user
make_binary_label("single", directory, 1, []) # choosing the user





data_single = pd.read_csv("temp/data_auth_single.csv", header=None)
label_single = pd.read_csv("temp/label_auth_single.csv", header=None, usecols=[0])


data_all = pd.read_csv("temp/data_auth_all.csv", header=None)
label_all = pd.read_csv("temp/label_auth_all.csv", header=None)


label_spoilt = spoil_data(label_single, 0.1)

data_train, label_train = data_single.values, label_spoilt.values
data_test, label_test = data_all.values, label_all.values

###############
#     TREE    #
###############
 
#data_train, data_test, label_train, label_test = train_test_split(data_all.values, label_all.values, test_size=0.3,random_state=17)


tree = DecisionTreeClassifier(max_depth=5, random_state=17)
tree.fit(data_train, label_train)
tree_pred = tree.predict(data_test)
acc_tree = accuracy_score(label_test, tree_pred) 
print('\n########## TREE #########')
#print('Score: ', acc_tree)



############################
#Parameter tuning for tree #
############################

tree_params = {'max_depth': range(1,11), 'max_features': range(4,9)}
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)


X_train2, y_train2 = data_single.values, label_spoilt[0]
X_holdout2, y_holdout2 = data_all.values, label_all[0]

#X_train2, X_holdout2, y_train2, y_holdout2 = train_test_split(data_all.values, label_all[0], test_size=0.3,
#random_state=17)
tree_grid.fit(X_train2, y_train2)

tr_bst_prm = tree_grid.best_params_ #Лучшее сочетание параметров и соответствующая средняя доля правильных ответов на кросс-валидации:
tr_bst_scr = tree_grid.best_score_
tr_acc_grd = accuracy_score(y_holdout2, tree_grid.predict(X_holdout2))


print ('Best parameters:', tr_bst_prm)
print('Best score: ', tr_bst_scr)



###############
#     KNN     #
###############

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(data_train, label_train)
knn_pred = knn.predict(data_test)
acc_knn = accuracy_score(label_test, knn_pred) 

print('\n########## KNN #########')
#print('Score: ', acc_knn)


############################
#Parameter tuning for KNN  #
############################

knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])
knn_params = {'knn__n_neighbors': range(1, 10)}

X_train3, y_train3 = data_single.values, label_spoilt[0]
X_holdout3, y_holdout3 = data_all.values, label_all[0]

#X_train3, X_holdout3, y_train3, y_holdout3 = train_test_split(data_all.values, label_all[0], test_size=0.3,
#random_state=17)


knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(X_train3, y_train3)

knn_bst_prm = knn_grid.best_params_
knn_bst_scr = knn_grid.best_score_
knn_acc_grd = accuracy_score(y_holdout3, knn_grid.predict(X_holdout3))



print ('Best parameters:', knn_bst_prm)
print('Best score: ', knn_bst_scr)



###############
#    ELM      #
###############
print('\n########## ELM #########')

def make_classifiers(nh):
    names = ["ELM(tanh)", "ELM(tanh,LR)", "ELM(sinsq)", "ELM(tribas)", "ELM(hardlim)", "ELM(rbf(0.1))"]
    

    # pass user defined transfer func
    sinsq = (lambda x: np.power(np.sin(x), 2.0))
    srhl_sinsq = MLPRandomLayer(n_hidden=nh, activation_func=sinsq)

    # use internal transfer funcs
    srhl_tanh = MLPRandomLayer(n_hidden=nh, activation_func='tanh')
    srhl_tribas = MLPRandomLayer(n_hidden=nh, activation_func='tribas')
    srhl_hardlim = MLPRandomLayer(n_hidden=nh, activation_func='hardlim')

    # use gaussian RBF
    srhl_rbf = RBFRandomLayer(n_hidden=nh*2, rbf_width=0.1, random_state=0)
    log_reg = LogisticRegression()

    classifiers = [GenELMClassifier(hidden_layer=srhl_tanh),
                   GenELMClassifier(hidden_layer=srhl_tanh, regressor=log_reg),
                   GenELMClassifier(hidden_layer=srhl_sinsq),
                   GenELMClassifier(hidden_layer=srhl_tribas),
                   GenELMClassifier(hidden_layer=srhl_hardlim),
                   GenELMClassifier(hidden_layer=srhl_rbf)]

    return names, classifiers




max_score = 0

for nh in range (10, 100):
    names, classifiers = make_classifiers(nh)
    for elm_name, elm_clf in zip(names, classifiers):
            #data_train_elm, data_test_elm, label_train_elm, label_test_elm = train_test_split(data_all.values, label_all.values, test_size=0.3,random_state=17)
            data_train_elm, label_train_elm = data_single.values, label_spoilt[0]
            data_test_elm, label_test_elm = data_all.values, label_all[0]

            elm_clf.fit(data_train_elm, label_train_elm)
            elm_score = elm_clf.score(data_test_elm, label_test_elm)
            if (elm_score > max_score):
                max_score = elm_score
                max_nh = nh
                max_elm_name = elm_name
            #print('Model %s score: %s' % (elm_name, elm_score))
print(max_score, max_elm_name, max_nh)




###############
#    SVM      #
###############

#data_train_svm, data_test_svm, label_train_svm, label_test_svm = train_test_split(data_all.values, label_all.values, test_size=0.3,
#random_state=17)


data_train_svm, label_train_svm = data_single.values, label_spoilt[0]
data_test_svm, label_test_svm = data_all.values, label_all[0]


clf = LinearSVC(random_state=0)
clf.fit(data_train_svm, label_train_svm)

clf_pred = clf.predict(data_test_svm)
acc_clf = accuracy_score(label_test_svm, clf_pred) 

print('\n########## SVM #########')
print('Score: ', acc_clf)
print('SVM-coefficient:')
print(clf.coef_)


###############
#    ANN      #
###############



print('\n########## ANN #########')
#data_train_ann, data_test_ann, label_train_ann, label_test_ann = train_test_split(data_all.values, label_all.values, test_size=0.3,
#random_state=17)


data_train_ann, label_train_ann = data_single.values, label_spoilt[0]
data_test_ann, label_test_ann = data_all.values, label_all[0]


max_sc_ann = 0

for ls2 in range (10,40):
    for ls1 in range(8, 20, 2):
        ann_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(ls1, ls2), random_state=1)
        ann_clf.fit(data_train_ann, label_train_ann)      
        ann_clf_pred = ann_clf.predict(data_test_ann)

        acc_ann = accuracy_score(label_test_ann, ann_clf_pred) 
        if (acc_ann > max_sc_ann):
            max_sc_ann = acc_ann
            max_ls1 = ls1
            max_ls2 = ls2
                                
print('Score (', max_ls1, max_ls2, 'layers): ', max_sc_ann)




########## TREE #########
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.7s finished


('Best parameters:', {'max_features': 4, 'max_depth': 1})
('Best score: ', 0.90090090090090091)

########## KNN #########
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    3.1s finished


('Best parameters:', {'knn__n_neighbors': 7})
('Best score: ', 0.90090090090090091)

########## ELM #########
(0.44606003752345214, 'ELM(rbf(0.1))', 98)

########## SVM #########
('Score: ', 0.18058161350844279)
SVM-coefficient:
[[ 0.44536923 -0.33381518 -0.01556905  0.19590321  0.12624738  0.06933421
  -0.07148316 -0.14921394  0.46417226 -0.46217992  0.22840538 -0.07594648
  -0.07276618  0.31606644 -0.09257216  0.06227642 -0.11676871 -0.42863163
  -0.02068629  0.24361045 -0.35968769  0.01522314 -0.45583473  0.03461724
  -0.08626043  0.42859165  0.42517002  0.14132674 -0.38848569  0.16604493
  -0.16530289  0.05615412 -0.12094207  0.31624641  0.04040588 -0.40193796]]

########## ANN #########
('Score (', 16, 17, 'layers): ', 0.39305816135084426)


# 